<a href="https://colab.research.google.com/github/Gyanma-rev/Car_Sales/blob/main/Car_Seller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing


Import libraries and dataset

In [ ]:
#  Import libraries and dataset
import pandas as pd
import numpy as np

import re
from datetime import datetime

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/ICon_Car_Seller/Datasets/original/corrected_original_transaction_list.csv", sep=',')

Check for missing data

In [ ]:
data.isnull().sum().value_counts()

In [ ]:
data.isnull().sum(axis=1).value_counts()

Drop instances with missing values

In [ ]:
data = data.dropna()

Remove duplicates

In [ ]:
num_duplicates = data.duplicated().sum()
print("Number of duplicates:", num_duplicates)

Number of duplicates: 0


In [ ]:
data = data.drop_duplicates()

Corrections of features values

In [ ]:
data['color'] = data['color'].replace('—', 'unknown')
data['interior'] = data['interior'].replace('—', 'unknown')

data['make'] = data['make'].astype(str).apply(lambda x: x.lower().replace(' ', '-').replace(',', '').replace('"', '').replace('\'', ''))
data['model'] = data['model'].astype(str).apply(lambda x: x.lower().replace(' ', '-').replace(',', '').replace('"', '').replace('\'', ''))
data['trim'] = data['trim'].astype(str).apply(lambda x: x.lower().replace(' ', '-').replace(',', '').replace('"', '').replace('\'', ''))
data['body'] = data['body'].astype(str).apply(lambda x: x.lower().replace(' ', '-').replace(',', '').replace('"', '').replace('\'', ''))
data['seller'] = data['seller'].astype(str).apply(lambda x: x.lower().replace(' ', '-').replace(',', '').replace('"', '').replace('\'', ''))


Add Transaction_ID column

In [ ]:
    # Generate a randomized sequence of IDs equal to the number of rows in the DataFrame
id_values = np.random.choice(range(100000, 999999 + 1), size=len(data), replace=False)

    # Assign the randomized IDs to a new 'ID' column in the DataFrame
data['transaction_ID'] = id_values

Change date format into integer

In [ ]:
def convert_to_integer(date_str):
    # Define the regex pattern to capture the month, day, and year
    pattern = r'^[A-Za-z]{3} ([A-Za-z]{3}) (\d{2}) (\d{4})'

    # Attempt to match the pattern in the date string
    match = re.search(pattern, date_str)
    if match:
        # Extract the month, day, and year
        month_str, day, year = match.groups()

        # Map month abbreviations to numbers
        month_mapping = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
            'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
            'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        # Convert month string to number
        month = month_mapping.get(month_str, 0)

        # Calculate the difference and convert to seconds

        delta = datetime(int(year), month, int(day)) - datetime(1970, 1, 1)
        return delta.days

    # Return None if the format doesn't match
    return None

def calculate_delta(date_str):

    delta = datetime(int(date_str), 1, 1) - datetime(1970, 1, 1)
    return delta.days

data['saledate'] = data['saledate'].apply(convert_to_integer)
data['year'] = data['year'].apply(calculate_delta)



Obtain statistical data




In [ ]:
## Count the number of rows and columns

print(f"The number of rows in the DataFrame is: {len(data)}")
print(f"The number of columns in the DataFrame is: {len(data.columns)}")


## Get Mode

for column in data.columns:
    mode = data[column].value_counts().head(5)
    unique_count = data[column].nunique()
    print("Feature:", column)
    print(mode)
    print("Number of unique values:", unique_count)
    print()




## Get Statistical measures

data.describe()

The number of rows in the DataFrame is: 472358
The number of columns in the DataFrame is: 17
Feature: year
year
15706    87474
15340    87382
16071    69714
14975    41384
13879    27011
Name: count, dtype: int64
Number of unique values: 26

Feature: make
make
ford         81014
chevrolet    54150
nissan       44043
toyota       35313
dodge        27183
Name: count, dtype: int64
Number of unique values: 53

Feature: model
model
altima    16346
fusion    12117
f-150     11950
camry     10986
escape    10656
Name: count, dtype: int64
Number of unique values: 764

Feature: trim
trim
base       48782
se         39635
lx         18707
limited    15498
lt         15273
Name: count, dtype: int64
Number of unique values: 1481

Feature: body
body
sedan        211330
suv          120968
hatchback     23821
minivan       21938
coupe         15960
Name: count, dtype: int64
Number of unique values: 44

Feature: transmission
transmission
automatic    455996
manual        16362
Name: count, dtype: in

,year,condition,odometer,mmr,sellingprice,saledate,transaction_ID
count,472358.000000,472358.000000,472358.000000,472358.000000,472358.000000,472358.000000,472358.000000
mean,14687.047405,3.426577,66698.386357,13836.991487,13690.340606,16501.216171,549967.662339
std,1396.087092,0.943685,51939.479228,9531.910071,9612.755824,59.106080,259837.864126
min,7305.000000,1.000000,1.000000,25.000000,1.000000,16071.000000,100000.000000
25%,13879.000000,2.700000,28134.000000,7425.000000,7200.000000,16457.000000,324875.250000
50%,15340.000000,3.600000,51081.000000,12300.000000,12200.000000,16483.000000,550191.000000
75%,15706.000000,4.200000,96588.000000,18300.000000,18200.000000,16576.000000,774990.250000
max,16436.000000,5.000000,999999.000000,182000.000000,230000.000000,16637.000000,999995.000000


Print the dataframe in a cvs


In [ ]:
data.to_csv("/content/drive/MyDrive/ICon_Car_Seller/Datasets/transaction_list.csv", index=False)

# Building the Knowledge Base

Import Dataset and libraries

In [ ]:
import pandas as pd
import numpy as np

!pip install pyswip
!apt-get install -y swi-prolog

from pyswip import Prolog

data = pd.read_csv("/content/drive/MyDrive/ICon_Car_Seller/Datasets/transaction_list.csv", sep=',')


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swi-prolog is already the newest version (8.4.2+dfsg-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Ulterior data preprocessing to better accomodate for prolog language

In [ ]:
##Add most_recent column

sorted_df = data[['transaction_ID', 'vin', 'saledate']].sort_values(by=['vin', 'saledate'], ascending=[True, False])
sorted_df['most_recent'] = ~sorted_df.duplicated(subset=['vin'], keep='first')
sorted_df['most_recent'] = sorted_df['most_recent'].map({True: 'true', False: 'false'})
data = pd.merge(data, sorted_df[['transaction_ID', 'most_recent']], on='transaction_ID', how='inner')
data.to_csv("/content/drive/MyDrive/ICon_Car_Seller/Datasets/transaction_list_with_most_recent.csv", index=False)

In [ ]:
## Add apices to values

data['vin'] = data['vin'].apply(lambda x: f"'{x}'")
data['model'] = data['model'].apply(lambda x: f"'{x}'")
data['trim'] = data['trim'].apply(lambda x: f"'{x}'")
data['make'] = data['make'].apply(lambda x: f"'{x}'")
data['body'] = data['body'].apply(lambda x: f"'{x}'")
data['transmission'] = data['transmission'].apply(lambda x: f"'{x}'")
data['state'] = data['state'].apply(lambda x: f"'{x}'")
data['color'] = data['color'].apply(lambda x: f"'{x}'")
data['interior'] = data['interior'].apply(lambda x: f"'{x}'")
data['seller'] = data['seller'].apply(lambda x: f"'{x}'")
data['most_recent'] = data['most_recent'].apply(lambda x: f"'{x}'")
data.to_csv("/content/drive/MyDrive/ICon_Car_Seller/Datasets/transaction_list_for_KB.csv", index=False)

Facts Description

In [ ]:
import pandas as pd

    ##car(Vin, Make, Model, Year, Body, Transmission)
    ##car_sold(Auction, Vin)
    ##auction(Auction, Seller, State, Price, Date, Condition, Odometer, Color, Interior, MMR, Recent)

datasets = [data[['vin', 'make', 'model', 'year', 'body', 'transmission']].drop_duplicates(),
             data[['transaction_ID', 'vin']].drop_duplicates(),
            data[['transaction_ID', 'seller', 'state', 'sellingprice', 'saledate', 'condition', 'odometer', 'color', 'interior', 'mmr', 'most_recent']].drop_duplicates()]

fact_names = ['car',
              'car_sold',
              'auction']

with open('/content/drive/MyDrive/ICon_Car_Seller/Prolog/facts.pl', 'w') as file:

    for df, fact_name in zip(datasets, fact_names):
        for _, row in df.iterrows():

            fact = f"{fact_name}({', '.join(map(str, row.values))}).\n"


            file.write(fact)



Rules

In [ ]:
with open('/content/drive/MyDrive/ICon_Car_Seller/Prolog/rules.pl', 'w') as file:

    ##car(Vin, Make, Model, Year, Body, Transmission)
    ##car_sold(Auction, Vin)
    ##auction(Auction, Seller, State, Price, Date, Condition, Odometer, Color, Interior, MMR, Recent)

    file.write('was_sold_repeatedly(Car):- car_sold(Auction, Car), car_sold(Auction2, Car), Auction \\== Auction2.\n')
    file.write('was_sold_within_one_year(Car):- car_sold(Auction, Car), car(Vin, _, _, ProdYear, _, _), auction(Auction, _, _, _, AuctYear, _ , _, _, _, _, _), Delta is AuctYear - ProdYear, Delta < 365.\n')

    file.write('most_recent_sale(Car, Auction) :- car_sold(Auction, Car), auction(Auction, _, _, _, _, _, _, _, _, _, Recent), Recent == true.\n')
    file.write('current_car_color(Car, Color):- most_recent_sale(Car, Auction), auction(Auction, _, _, _, _, _, _, Color, _, _, _).\n')
    file.write('current_car_interior(Car, Interior):- most_recent_sale(Car, Auction), auction(Auction, _, _, _, _, _, _, _, Interior, _, _).\n')
    file.write('current_car_mmr(Car, MMR):- most_recent_sale(Car, Auction), auction(Auction, _, _, _, _, _, _, _, _, MMR, _).\n')
    file.write('current_car_selling_price(Car, Price):- most_recent_sale(Car, Auction), auction(Auction, _, _, Price, _, _, _, _, _, _, _).\n')
    file.write('current_car_condition(Car, Condition):- most_recent_sale(Car, Auction), auction(Auction, _, _, _, _, Condition, _, _, _, _, _).\n')
    file.write('current_car_odometer(Car, Odometer):- most_recent_sale(Car, Auction), auction(Auction, _, _, _, _, _, Odometer, _, _, _, _).\n')
    file.write('last_seller(Car, Seller):- most_recent_sale(Car, Auction), auction(Auction, Seller, _, _, _, _, _, _, _, _, _).\n')
    file.write('does_color_match_interior(Car):- current_car_color(Car, Color), current_car_interior(Car, Interior), Color == Interior.\n')
    file.write('was_sold_above_mmr(Car):- current_car_selling_price(Car, SP), current_car_mmr(Car, MP), SP > MP.\n')
    file.write('was_sold_below_mmr(Car):- current_car_selling_price(Car, SP), current_car_mmr(Car, MP), SP < MP.\n')
    file.write('was_sold_at_mmr(Car):- current_car_selling_price(Car, SP), current_car_mmr(Car, MP), SP == MP.\n')
    file.write('is_same_color(Car1, Car2):- Car1 \== Car2, current_car_color(Car1, Color), current_car_color(Car2, Color).\n')
    file.write('is_same_interior(Car1, Car2):- Car1 \== Car2, current_car_interior(Car1, Interior), current_car_interior(Car2, Interior).\n')
    file.write('is_same_market_price(Car1, Car2):- Car1 \== Car2, current_car_mmr(Car1, MMR), current_car_mmr(Car2, MMR).\n')
    file.write('is_same_selling_price(Car1, Car2):- Car1 \== Car2, current_car_selling_price(Car1, Price), current_car_selling_price(Car2, Price).\n')
    file.write('is_same_condition(Car1, Car2):- Car1 \== Car2, current_car_condition(Car1, Condition), current_car_condition(Car2, Condition).\n')
    file.write('is_same_odometer(Car1, Car2):- Car1 \== Car2, current_car_odometer(Car1, Odometer), current_car_odometer(Car2, Odometer).\n')

    file.write('is_same_make(Car1, Car2) :- Car1 \== Car2, car(Car1, Make, _, _, _, _), car(Car2, Make, _, _, _, _).\n')

    file.write('is_same_production_year(Car1, Car2):- Car1 \== Car2, car(Car1, _, _, Year, _, _), car(Car2, _, _, Year, _, _).\n')
    file.write('is_same_body(Car1, Car2):- Car1 \== Car2, car(Car1, _, _, _, Body, _), car(Car2, _, _, _, Body, _).\n')
    file.write('is_same_transmission(Car1, Car2):- Car1 \== Car2, car(Car1, _, _, _, _, Transmission), car(Car2, _, _, _, _, Transmission).\n')

    file.write('high_volume_seller(Seller):- findall(Auction, auction(Auction, Seller, _, _, _, _, _, _, _, _, _), List), length(List, N), N > 200.\n')
    file.write('highly_traded_make(Make):- findall(Auction, (car_sold(Auction, Car), car(Car, Make, _, _, _, _)), List), length(List, N), N > 1000.')


Example queries

In [ ]:
with open('/content/drive/MyDrive/ICon_Car_Seller/Prolog/query_inputs.txt', 'w') as file:
    file.write('wba6b2c57ed129731,')
    file.write('wp0ca2988xu629622,')
    file.write('2c4rdgeg8er151143,')
    file.write('wbayp9c59ed169280,')
    file.write('5npeb4acxdh744455')

In [ ]:
from pyswip import Prolog
prolog = Prolog()

prolog.consult('/content/drive/MyDrive/ICon_Car_Seller/Prolog/rules.pl')
prolog.consult('/content/drive/MyDrive/ICon_Car_Seller/Prolog/facts.pl')

In [ ]:
input_file = open('/content/drive/MyDrive/ICon_Car_Seller/Prolog/query_inputs.txt', "r")

cars_queried = input_file.read().split(",")

input_file.close

with open('/content/drive/MyDrive/ICon_Car_Seller/Prolog/query_outputs.txt', "w") as file:

    for car in cars_queried:

        file.write(f"Car: {car}\n")
        file.write(f"\n")

        ##Was sold repeatedly
        if list(prolog.query(f"was_sold_repeatedly('{car}')")):
            file.write(f"{car} was sold repeatedly\n")
        else:
            file.write(f"{car} was not sold repeatedly\n")

        ##Was sold within one year
        file.write(f"\n")
        if list(prolog.query(f"was_sold_within_one_year('{car}')")):
            file.write(f"{car} was sold within one year\n")
        else:
            file.write(f"{car} was not sold within one year\n")

        ##Was sold above MMR
        file.write(f"\n")
        if list(prolog.query(f"was_sold_above_mmr('{car}')")):
            file.write(f"{car} was sold above its MMR\n")
        else:
            if list(prolog.query(f"was_sold_below_mmr('{car}')")):
                file.write(f"{car} was sold below its MMR\n")
            else:
                if list(prolog.query(f"was_sold_at_mmr('{car}')")):
                    file.write(f"{car} was sold at its MMR\n")
                else:
                    file.write(f"Couldn't find selling info about {car}\n")


        ##Most recent sale
        file.write(f"\n")
        if list(prolog.query(f"most_recent_sale('{car}', Auction)")):
            recent_sale_result = list(prolog.query(f"most_recent_sale('{car}', Auction)"))
            recent_sale = str(recent_sale_result[0]['Auction'])
            file.write(f"{car}\'s most recent sale code is:\n")
            file.write(recent_sale)
            file.write(f"\n")
        else:
            file.write(f"Couldn't find {car}\'s most recent sale code.\n")

        if list(prolog.query(f"last_seller('{car}', Seller)")):
            seller_result = list(prolog.query(f"last_seller('{car}', Seller)"))
            seller = str(seller_result[0]['Seller'])
            file.write(f"{car}\'s most recent seller is:\n")
            file.write(seller)
            file.write(f"\n")
            if list(prolog.query(f"high_volume_seller('{seller}')")):
               file.write(f"{seller} is a high volume seller.\n")
            else:
               file.write(f"{seller} is not a high volume seller.\n")
        else:
             file.write(f"Couldn't find {car}\'s most recent seller.\n")




        ##its make is highly traded
        file.write(f"\n")
        if list(prolog.query(f"car('{car}', Make, _, _, _, _)")):
            result = list(prolog.query(f"car('{car}', Make, _, _, _, _)"))
            make = str(result[0]['Make'])
            file.write(f"{car}\'s make is:\n")
            file.write(make)
            file.write(f"\n")
            if list(prolog.query(f"highly_traded_make('{make}')")):
              file.write(f"{make} is a highly traded make.\n")
            else:
              file.write(f"{make} is not a highly traded make.\n")
        else:
            file.write(f"Couldn't find {car}\'s make.\n")

        ##Current car color
        file.write(f"\n")
        if list(prolog.query(f"current_car_color('{car}', Color)")):
            result = list(prolog.query(f"current_car_color('{car}', Color)"))
            color = str(result[0]['Color'])
            file.write(f"{car}\'s current color is:\n")
            file.write(color)
            file.write(f"\n")
        else:
            file.write(f"Couldn't find {car}\'s most recent color.\n")

        ##Current car interior
        file.write(f"\n")
        if list(prolog.query(f"current_car_interior('{car}', Color)")):
            result = list(prolog.query(f"current_car_interior('{car}', Color)"))
            color = str(result[0]['Color'])
            file.write(f"{car}\'s current interior color is:\n")
            file.write(color)
            file.write(f"\n")
        else:
            file.write(f"Couldn't find {car}\'s most recent interior color .\n")


        ##Does color match interior
        file.write(f"\n")
        if list(prolog.query(f"does_color_match_interior('{car}')")):
            file.write(f"{car}\'s color matches its interior color.\n")
        else:
            file.write(f"{car}\'s color doesn't match its interior color.\n")


        ##Current car MMR
        file.write(f"\n")
        if list(prolog.query(f"current_car_mmr('{car}', MMR)")):
            result = list(prolog.query(f"current_car_mmr('{car}', MMR)"))
            mmr = str(result[0]['MMR'])
            file.write(f"{car}\'s current MMR is:\n")
            file.write(mmr)
        else:
            file.write(f"Couldn't find {car}\'s current MMR.\n")


        ##Current car selling price
        file.write(f"\n")
        if list(prolog.query(f"current_car_selling_price('{car}', Price)")):
            result = list(prolog.query(f"current_car_selling_price('{car}', Price)"))
            price = str(result[0]['Price'])
            file.write(f"{car}\'s current selling price is:\n")
            file.write(price)
            file.write(f"\n")
        else:
            file.write(f"Couldn't find {car}\'s current selling price.\n")


        ##Current car condition
        file.write(f"\n")
        if list(prolog.query(f"current_car_condition('{car}', Condition)")):
            result = list(prolog.query(f"current_car_condition('{car}', Condition)"))
            condition = str(result[0]['Condition'])
            file.write(f"{car}\'s current condition is:\n")
            file.write(condition)
            file.write(f"\n")
        else:
            file.write(f"Couldn't find {car}\'s current condition.\n")

        ##Current car odometer
        file.write(f"\n")
        if list(prolog.query(f"current_car_odometer('{car}', Odometer)")):
            result = list(prolog.query(f"current_car_odometer('{car}', Odometer)"))
            odometer = str(result[0]['Odometer'])
            file.write(f"{car}\'s current odometer is:\n")
            file.write(odometer)
            file.write(f"\n")
        else:
            file.write(f"Couldn't find {car}\'s current odometer.\n")

        ##Comparisons
        file.write(f"\n")
        for new_car in cars_queried:
            if car != new_car:

                ##same make
                file.write(f"\n")
                if list(prolog.query(f"is_same_make('{car}', '{new_car}')")):
                    file.write(f"{car} is the same make as {new_car}\n")
                else:
                    file.write(f"{car} is not the same make as {new_car}\n")

                ##same color
                file.write(f"\n")
                if list(prolog.query(f"is_same_color('{car}', '{new_car}')")):
                    file.write(f"{car} is the same color as {new_car}\n")
                else:
                    file.write(f"{car} is not the same color as {new_car}\n")

                ##same interior
                file.write(f"\n")
                if list(prolog.query(f"is_same_interior('{car}', '{new_car}')")):
                    file.write(f"{car} is the same interior color as {new_car}\n")
                else:
                    file.write(f"{car} is not the same interior color as {new_car}\n")

                ##same market price
                file.write(f"\n")
                if list(prolog.query(f"is_same_market_price('{car}', '{new_car}')")):
                    file.write(f"{car} is the same market price as {new_car}\n")
                else:
                    file.write(f"{car} is not the same market price as {new_car}\n")


                ##same selling price
                file.write(f"\n")
                if list(prolog.query(f"is_same_selling_price('{car}', '{new_car}')")):
                    file.write(f"{car} is the same selling price as {new_car}\n")
                else:
                    file.write(f"{car} is not the same selling price as {new_car}\n")


                ##same condition
                file.write(f"\n")
                if list(prolog.query(f"is_same_condition('{car}', '{new_car}')")):
                    file.write(f"{car} is the same condition as {new_car}\n")
                else:
                    file.write(f"{car} is not the same condition as {new_car}\n")


                ##same odometer
                file.write(f"\n")
                if list(prolog.query(f"is_same_odometer('{car}', '{new_car}')")):
                    file.write(f"{car} is the same odometer as {new_car}\n")
                else:
                    file.write(f"{car} is not the same odometer as {new_car}\n")


                ##same production year
                file.write(f"\n")
                if list(prolog.query(f"is_same_production_year('{car}', '{new_car}')")):
                    file.write(f"{car} is the same production year as {new_car}\n")
                else:
                    file.write(f"{car} is not the same production year as {new_car}\n")


                ##same body
                file.write(f"\n")
                if list(prolog.query(f"is_same_body('{car}', '{new_car}')")):
                    file.write(f"{car} is the same body as {new_car}\n")
                else:
                    file.write(f"{car} is not the same body as {new_car}\n")


                ##same transmission
                file.write(f"\n")
                if list(prolog.query(f"is_same_transmission('{car}', '{new_car}')")):
                    file.write(f"{car} is the same transmission as {new_car}\n")
                else:
                    file.write(f"{car} is not the same transmission as {new_car}\n")
                file.write(f"\n")


        file.write(f"\n")
        file.write(f"\n")
        file.write(f"\n")
        file.write(f"\n")


# Machine Learning

Creating the "car" dataset with the data available for each car from the last registered sale

In [8]:
import pandas as pd


data = pd.read_csv("/content/drive/MyDrive/ICon_Car_Seller/Datasets/transaction_list_with_most_recent.csv", sep=',')

data = data[data['most_recent'] == True]

data = data[['make', 'model', 'year', 'trim' , 'body', 'transmission', 'seller', 'sellingprice', 'condition', 'odometer', 'color', 'interior', 'mmr']]

data.to_csv("/content/drive/MyDrive/ICon_Car_Seller/Datasets/car_dataset.csv", index=False)

#### K-Means

Elbow Method Application

In [2]:
pip install kmodes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from kmodes.kmodes import KModes


def calculate_elbow (seed):

    # Range of k values to test
    k_values = range(1, 8)

    # List to hold the cost values for each k
    cost_values = []

    # Calculate K-Modes for each k and store the cost
    for k in k_values:
        kmodes = KModes(n_clusters=k, init='Cao', n_init=5, random_state=0)
        kmodes.fit(data)
        cost_values.append(kmodes.cost_)

    # Calculate and print the slope for each step
    print("Number of Clusters (k) | Cost | Slope | Absolute Reduction | Percentage Reduction")
    for i in range(1, len(cost_values)):
        slope = cost_values[i] - cost_values[i - 1]
        absolute_reduction = abs(slope)
        percentage_reduction = (absolute_reduction / cost_values[i - 1]) * 100

        print(f"{k_values[i]:>18} | {cost_values[i]:>5} | {slope:>6.2f} |"
              f" {absolute_reduction:>17.2f} | {percentage_reduction:>19.2f}%")
    # Plot the results
    plt.figure(figsize=(8, 5))
    plt.plot(k_values, cost_values, 'bo-')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Cost (Total Dissimilarity)')
    plt.title('Elbow Method for Optimal k with K-Modes')
    plt.xticks(k_values)
    plt.show()


calculate_elbow (data)
plt.savefig("/content/drive/MyDrive/ICon_Car_Seller/K-Means/elbow_method_k_modes.png", format="png", dpi=300)  # Save the plot

